In [ ]:
# pip install ipywidgets

In [1]:
import torch
import numpy as np
import json

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)


In [2]:
with open("../nlu_engine/intents.json", "r", encoding="utf-8") as f:
    data = json.load(f)

texts = []
labels = []

label2id = {intent["name"]: i for i, intent in enumerate(data["intents"])}
id2label = {i: name for name, i in label2id.items()}

for intent in data["intents"]:
    for example in intent["examples"]:
        texts.append(example)
        labels.append(label2id[intent["name"]])


In [3]:
print(f"Total examples: {len(texts)}")
display(texts[:5])

Total examples: 80


["What's my account balance?",
 'Show balance for my savings account',
 'How much money do I have in my current account?',
 'Check balance of my savings',
 'Can you tell me my account balance?']

In [4]:
print(f"Total labels: {len(set(labels))}")

Total labels: 4


In [5]:
shown_labels = len(label2id)
print(f"Shown labels: {shown_labels}")
print(f"Label to ID mapping: {label2id}")
print(f"ID to Label mapping: {id2label}")

Shown labels: 4
Label to ID mapping: {'check_balance': 0, 'transfer_money': 1, 'card_block': 2, 'find_atm': 3}
ID to Label mapping: {0: 'check_balance', 1: 'transfer_money', 2: 'card_block', 3: 'find_atm'}


In [6]:
train_texts, val_texts, train_labels, val_labels = train_test_split(texts,labels,test_size=0.2,random_state=42)


In [7]:
MODEL_NAME = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [8]:
def tokenize(texts, labels):
    encodings = tokenizer(
        texts,
        truncation=True,
        padding=True,
        max_length=128,
        return_tensors="pt"
    )
    encodings["labels"] = torch.tensor(labels)
    return encodings


In [9]:
class IntentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings["labels"])

    def __getitem__(self, idx):
        return {k: v[idx] for k, v in self.encodings.items()}


In [11]:
train_encodings = tokenize(train_texts, train_labels)
val_encodings = tokenize(val_texts, val_labels)

train_dataset = IntentDataset(train_encodings)
val_dataset = IntentDataset(val_encodings)


In [12]:
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(label2id)
)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
training_args = TrainingArguments(
    output_dir="../models/intent_model_2",
    num_train_epochs=8,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,  # 0.00002
    logging_dir="../models/intent_model_2/logs",
    do_eval=True,
    save_steps=500
)


In [14]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)

    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1": f1_score(labels, predictions, average="weighted")
    }


In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

C:\Users\admin\AppData\Local\Temp\ipykernel_8576\2721217396.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [16]:
trainer.train()

d:\Mentor\BankBot_AI\bankbot_ai\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss


TrainOutput(global_step=64, training_loss=0.9449846744537354, metrics={'train_runtime': 86.4921, 'train_samples_per_second': 5.92, 'train_steps_per_second': 0.74, 'total_flos': 2384498221056.0, 'train_loss': 0.9449846744537354, 'epoch': 8.0})

In [18]:
def predict_intent(text):
    inputs = tokenizer(text, return_tensors="pt")

    with torch.no_grad():
        outputs = model(**inputs)

    probs = torch.softmax(outputs.logits, dim=1)
    pred_id = torch.argmax(probs).item()

    return {
        "intent": id2label[pred_id],
        "confidence": probs[0][pred_id].item()
    }


In [19]:
predict_intent("I want to block my debit card")


{'intent': 'card_block', 'confidence': 0.5264736413955688}

In [20]:
predict_intent("Please transfer $250 to my friend")

{'intent': 'transfer_money', 'confidence': 0.295680433511734}

In [21]:
predict_intent("How can I open a new savings account?")

{'intent': 'check_balance', 'confidence': 0.3049277663230896}

In [22]:
predict_intent("transfer 500 dollars to john")

{'intent': 'transfer_money', 'confidence': 0.35596442222595215}